<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/3%D0%9A%D0%B5%D0%B9%D1%812_%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%B5__%D0%B2%D1%80%D0%B5%D0%BC%D1%8F_%D0%BD%D0%B0_%D1%83%D1%81%D0%BF%D0%B5%D1%88%D0%BD%D0%BE%D0%B5_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import csv
from datetime import datetime
from collections import defaultdict

def analyze(file_path) :
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)
        rows = [row for row in reader]
    if not rows:
        print("Файл пуст")
        return 0.0
    rows.sort(key=lambda x: (x[2], x[0], x[1])) # Сортировка по задачам

    prob_times=defaultdict(list)  # Словарь со списком решенных задач и времени на их решение (значения - списки)
    #   prob   время решения задач неск. пользователями
    # {'100': [109.119613, 57.990168, 29.308994], '101': [172.822919], '102': [56.022572, 2686.898138, 9.955443]})
    user_state=defaultdict(dict)  # Словарь пользователей, задач и статусов (значения - словари)
    #   user      prob            дата/время 1-й попытки                              тек. статус     статус первой попытки
    # {'12852': {'100': {'start': datetime.datetime(2023, 4, 30, 16, 2, 11, 285436), 'status': False, 'first': '0'}}})

    for row in rows:
        date=row[0]
        user=row[1]
        prob=row[2]
        stat=row[3]

        try:
            time = datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
        except Exception as e:
            print(f"Некорректная дата: {e}")
            continue

        # Если есть записи по решенной задачи в обоих словарях
        if prob in user_state[user] and user_state[user][prob].get('status'):
            continue
        # Запись в словарь пользователей
        if prob not in user_state[user]: # start - дата/время первой попытки, status - статус решения по умолчанию False,
                                         # first - результат первой попытки
            user_state[user][prob] = {'start':time,'status':False,'first':stat}

        if stat=='1': # Задача успешно решена
            if user_state[user][prob]['first']=='1':  # При успешном первом решении удаляем гениев
                user_state[user][prob]['status']=True
                continue
            date_time=(time-user_state[user][prob]['start']).total_seconds() # Разница во времени
            if date_time>0:
                prob_times[prob].append(date_time) # Добавление данных в словарь решенных задач
            user_state[user][prob]['status']=True  # Помечаем задачу как решенную для этого пользователя

    avg_time=[] # Список средних значений времени на решение всех задач
    for prob,times in prob_times.items():
        if times:
            avg=sum(times)/len(times) # Ищем среднее время решения для каждой задачи
            avg_time.append(avg) # Добавляем в список средних значений

    if avg_time:
        res=round(sum(avg_time)/len(avg_time),2) # Расчет среднего времени решения всех задач
        return res
    else:
        print("Нет подходящих решений для расчета")
        return 0.0

res=analyze('codesubmit.csv')
print(res)


611.86
